## 作業

In [3]:
import pandas
google = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx', index_col = 0)

In [4]:
google.head()

,username,score,comment
0,倪依芙,5,起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...
1,Mimi龍,4,聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...
2,林昌逸,1,非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡
3,林幸蓁,5,內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...
4,台灣TAXI市區叫車禮車包車約拍找小鄭,5,乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...


In [5]:
def convertScore(score):
  if score >= 4:
    return 'good'
  elif score <= 2:
    return 'bad'
  else: 
    return 'soso'

In [6]:
google['status'] = google['score'].map(convertScore)

In [7]:
import jieba
corpus = []
for e in google['comment'].tolist():
    corpus.append(list(jieba.cut(e)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.940 seconds.
Prefix dict has been built successfully.


In [8]:
from gensim.models import word2vec
size = 50  # 產生多少維度 
min_count = 5 # 要算至少出現多少次數的字詞
workers = -1 # 使用多少個core 計算, -1 使用所有的core 進行計算 
window = 10  # 上下文的區間
iter = 1000 # 神經網路訓練的迭代數
sample = 1e-5 # 取樣的數量
model = word2vec.Word2Vec(corpus, 
                          workers = workers,
                          sample = sample,
                          size = size,
                          min_count=min_count,
                          window = window,
                          iter = iter)

In [10]:
#for ele in model.wv.vocab:
#  print(ele)

In [11]:
model.wv.get_vector('越南')

array([-6.2473821e-03, -1.5602022e-03,  4.3593664e-03, -7.6216026e-03,
        3.2578735e-04, -3.8040073e-03,  4.6569351e-03, -5.2936017e-03,
       -8.0661001e-03, -1.8582003e-03, -5.2584196e-03,  5.7037883e-03,
       -4.8752283e-03, -2.6064257e-03,  7.5233249e-05,  8.6057857e-03,
        7.3851845e-03,  2.5813424e-03,  8.1432397e-03, -8.6519932e-03,
       -6.9618798e-03, -4.6876217e-03, -6.9317590e-03,  4.2128176e-04,
       -8.2872005e-04, -3.7717759e-03, -7.6840674e-03, -2.0504254e-03,
        7.5913146e-03, -7.0984615e-03,  6.1308695e-03,  4.3035331e-03,
       -9.7447345e-03,  6.2935022e-03, -4.6922767e-04, -2.8430040e-03,
        2.9132112e-03,  1.7843049e-03,  5.6017037e-03,  2.7697338e-03,
       -8.8316081e-03,  7.7389916e-03, -9.2706848e-03,  7.2830929e-03,
        8.9329835e-03,  8.8444920e-03,  2.1003755e-03,  4.8506825e-04,
        7.2138994e-03,  2.1585922e-03], dtype=float32)

In [12]:
import jieba
corpus = []
tags = []
for idx,rec in google[google['status'].isin(['good', 'bad'])].iterrows():
  corpus.append(list(jieba.cut(rec.get('comment'))))
  tags.append(rec.get('status'))

In [13]:
import numpy as np
x = []
y = []
size = 50
for tag, s in zip(tags, corpus):
    vec = np.zeros(size).reshape((1, size))
    cnt = 0
    for w in s:
        if w in model:
            vec += model.wv.get_vector(w)
            cnt += 1
    if cnt > 0:
      x.append(vec / cnt)
      y.append(tag)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


In [15]:
x = np.concatenate(x, axis = 0)

In [16]:
x.shape

(440, 50)

In [17]:
len(y)

440

In [19]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [20]:
train_x.shape

(352, 50)

In [21]:
test_x.shape

(88, 50)

In [23]:
len(train_y)

352

In [24]:
len(test_y)

88

In [29]:
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf')
clf.fit(train_x, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
pred_y = clf.predict(test_x)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.7045454545454546

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred_y)

array([[29, 11],
       [15, 33]])

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(train_x, train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [35]:
pred_y = clf.predict(test_x)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.625

## Selenium
- https://chromedriver.chromium.org/
- https://www.largitdata.com/course_list/15

In [37]:
! pip install selenium

     |████████████████████████████████| 911kB 4.5MB/s 


In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('/opt/local/common/chromedriver')
driver.get('https://www.google.com/search?rlz=1C5CHFA_enTW718TW718&ei=gYZPX4XfFMO4mAWk3I6gDQ&q=google+%E7%8E%89%E6%AC%A3%E7%8F%8D&oq=google+%E7%8E%89%E6%AC%A3%E7%8F%8D&gs_lcp=CgZwc3ktYWIQAzIFCCEQoAEyBQghEKABOgQIABAeOgYIABAIEB5Q5xhYv1dg5l1oEXAAeAGAAcMBiAHFDZIBBDE4LjOYAQCgAQGqAQdnd3Mtd2l6wAEB&sclient=psy-ab&ved=0ahUKEwjFo4yxs8rrAhVDHKYKHSSuA9QQ4dUDCA0&uact=5')

In [ ]:
driver.find_element_by_css_selector('a[data-async-trigger="reviewDialog"]').click()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source)

In [ ]:
for review in soup.select('.review-snippet'):
    print(review.text)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")